In [11]:
import torch
import torchvision
import torchvision.transforms as transforms

In [12]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_pool1=nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.conv_pool2=nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.fc1=nn.Sequential(
            nn.Linear(16*5*5,120),
            nn.ReLU()
        )

        self.fc2=nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU()
        )

        self.out=nn.Sequential(
            nn.Linear(84,10)
        )
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x=self.conv_pool1(x)
        x=self.conv_pool2(x)
        x=torch.flatten(x,1)
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.out(x)
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x


net = Net().to(device)

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [16]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.225
[1,  4000] loss: 1.908
[1,  6000] loss: 1.704
[1,  8000] loss: 1.600
[1, 10000] loss: 1.549
[1, 12000] loss: 1.485
[2,  2000] loss: 1.400
[2,  4000] loss: 1.384
[2,  6000] loss: 1.368
[2,  8000] loss: 1.328
[2, 10000] loss: 1.308
[2, 12000] loss: 1.297
[3,  2000] loss: 1.216
[3,  4000] loss: 1.217
[3,  6000] loss: 1.226
[3,  8000] loss: 1.183
[3, 10000] loss: 1.183
[3, 12000] loss: 1.187
[4,  2000] loss: 1.104
[4,  4000] loss: 1.108
[4,  6000] loss: 1.105
[4,  8000] loss: 1.131
[4, 10000] loss: 1.109
[4, 12000] loss: 1.101
[5,  2000] loss: 1.025
[5,  4000] loss: 1.067
[5,  6000] loss: 1.050
[5,  8000] loss: 1.051
[5, 10000] loss: 1.060
[5, 12000] loss: 1.036
Finished Training


In [17]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device),data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 60 %


In [1]:
import numpy as np
a=np.random.randn(3,1,3,3)*0.1
a

array([[[[-1.10330575e-02, -9.40900127e-03,  6.71464969e-02],
         [-4.23648978e-05,  6.20930370e-03,  1.21788512e-01],
         [-1.15542310e-02, -1.14374020e-01,  1.07908278e-01]]],


       [[[-9.01701295e-02,  2.76137056e-02,  1.03731514e-01],
         [ 1.05088497e-01, -9.13300053e-02, -2.17615250e-02],
         [ 2.78611559e-02, -5.54749983e-02,  3.76707369e-02]]],


       [[[-8.96395085e-02, -6.55099743e-02, -1.76069145e-01],
         [-4.08922855e-02,  1.14963443e-02,  6.82776114e-02],
         [-2.79348112e-03, -2.04745359e-02, -1.27895711e-01]]]])

In [2]:
a.shape

(3, 1, 3, 3)

In [3]:
b=np.array([[1,2,3],[4,5,6]])
b

array([[1, 2, 3],
       [4, 5, 6]])

In [4]:
b[:][0:3]

array([[1, 2, 3],
       [4, 5, 6]])